In [1]:
#install the libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install langdetect
!pip install nltk

     |████████████████████████████████| 10.1MB 5.5MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=d28d2f5c2b0e1e5b131538c001388494fe33f847991e6fe5de425087fa959aaa
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry
     |████████████████████████████████| 983kB 5.2MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=312db62167e0c2c7863110ece154da5e945b4fc9130ec929a4fcdbdee0b57bdc
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [2]:
#import libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('vader_lexicon')
nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#authentication for Twitter API
consumerKey = "MQGKyDun6PoyUHv03I79dKDMQ"
consumerSecret = "UuRtq7ZuZKaqyj8ArwwQTYaIyF0KtNLp8S516GQlgXrAKRhA5v"
accessToken = "1369959237762363394-SMyHYyvwVjJ0twaZGytVbf7sa5ZWRs"
accessTokenSecret = "m1wC95sYY04WeyCXpENUfcgUlDuVqQR83Awoh0CBeiVZw"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [4]:
#sentiment analysis
def percentage(part, whole):
  return 100*float(part)/float(whole)

keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = 100 #don't change this, limited number of tweets per month !!!

tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
  #print the text of the tweet
  print(tweet.text)
  tweet_list.append(tweet.text)
  analysis = TextBlob(tweet.text)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity

  if neg > pos:
    negative_list.append(tweet.text)
    negative += 1
  elif pos > neg:
    positive_list.append(tweet.text)
    positive += 1
  elif pos == neg:
    neutral_list.append(tweet.text)
    neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

Please enter keyword or hashtag to search: elon musk
RT @LewSpears: Liking Elon Musk doesn’t mean you are anything LIKE Elon Musk #SpearsReelTalk https://t.co/vvz2mwl3zt
@braveheart034 Hocam Elon Musk twitlerine bin basar çok güzel olmuş
Elon Musk mission is to leave a dying planet to live on a dead planet
Volkswagen's boss is adamant that the German giant can prevail in the electric-car race with Tesla. He may be right… https://t.co/D1P1Oqrict
RT @TeslaNY: Wedbush: “We view Guillen moving to the heavy truck frontier as a significant strategic move showing Tesla plans to double dow…
.@elonmusk turns down $1 million for the tweet he was selling as an NFT 

https://t.co/rwnBe7Pna4

By @pabsgill https://t.co/vscjml8ci0
RT @lingualandjp: イーロン・マスク氏が、自分の肩書きをCEOからテクノキングへ。孫正義氏が「お金大将」と名乗るようなものか。

https://t.co/zUH3ibIbk3
Liking Elon Musk doesn’t mean you are anything LIKE Elon Musk #SpearsReelTalk https://t.co/vvz2mwl3zt
RT @aliensosandy: elon musk and grimes baby when they ask it how to pronoun

In [6]:
#create data frames
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
tweet_list

,0
0,RT @LewSpears: Liking Elon Musk doesn’t mean y...
1,@braveheart034 Hocam Elon Musk twitlerine bin ...
2,Elon Musk mission is to leave a dying planet t...
3,Volkswagen's boss is adamant that the German g...
4,RT @TeslaNY: Wedbush: “We view Guillen moving ...
...,...
95,RT @PedroPorky15: Volkswagen CEO says he wants...
96,"Schlümm, dieses unerbittliche Tempo des Elon M..."
97,RT @balleralert: More Than 400 Workers At Tesl...
98,"eat that, elon musk"


In [7]:
#remove duplicated (if there are any)
tweet_list.drop_duplicates(inplace = True)
tweet_list

,0
0,RT @LewSpears: Liking Elon Musk doesn’t mean y...
1,@braveheart034 Hocam Elon Musk twitlerine bin ...
2,Elon Musk mission is to leave a dying planet t...
3,Volkswagen's boss is adamant that the German g...
4,RT @TeslaNY: Wedbush: “We view Guillen moving ...
...,...
93,RT @nizamettinhoca: Kumar böyle bir şey!\n\nTe...
94,Why elon musk trending do y'all not remember w...
95,RT @PedroPorky15: Volkswagen CEO says he wants...
96,"Schlümm, dieses unerbittliche Tempo des Elon M..."


In [8]:
#clean the text (RT, link, punctuation and convert to lowercase)
tw_list = pd.DataFrame(tweet_list)
tw_list['text'] = tw_list[0];

remove_RT = lambda x: re.sub("RT @\w+: ", " ", x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x)
tw_list['text'] = tw_list.text.map(remove_RT).map(rt)
tw_list['text'] = tw_list.text.map(rt)
tw_list['text'] = tw_list.text.str.lower()
tw_list.head(10)

,0,text
0,RT @LewSpears: Liking Elon Musk doesn’t mean y...,liking elon musk doesn t mean you are anythin...
1,@braveheart034 Hocam Elon Musk twitlerine bin ...,34 hocam elon musk twitlerine bin basar ok g...
2,Elon Musk mission is to leave a dying planet t...,elon musk mission is to leave a dying planet t...
3,Volkswagen's boss is adamant that the German g...,volkswagen s boss is adamant that the german g...
4,RT @TeslaNY: Wedbush: “We view Guillen moving ...,wedbush we view guillen moving to the heavy...
5,.@elonmusk turns down $1 million for the tweet...,turns down 1 million for the tweet he was ...
6,RT @lingualandjp: イーロン・マスク氏が、自分の肩書きをCEOからテクノキン...,ceo ...
7,Liking Elon Musk doesn’t mean you are anything...,liking elon musk doesn t mean you are anything...
8,RT @aliensosandy: elon musk and grimes baby wh...,elon musk and grimes baby when they ask it ho...
9,Coinvolgere come Elon Musk https://t.co/pbOaft...,coinvolgere come elon musk


In [9]:
#calculating negative, positive, neutral and compound values

tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
  score = SentimentIntensityAnalyzer().polarity_scores(row)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  if neg > pos:
    tw_list.loc[index, 'sentiment'] = 'negative'
  elif pos > neg:
    tw_list.loc[index, 'sentiment'] = 'positive'
  else:
    tw_list.loc[index, 'sentiment'] = 'neutral'
  tw_list.loc[index, 'neg'] = neg
  tw_list.loc[index, 'pos'] = pos
  tw_list.loc[index, 'neu'] = neu
  tw_list.loc[index, 'compound'] = comp

tw_list.head(10)

,0,text,polarity,subjectivity,sentiment,neg,pos,neu,compound
0,RT @LewSpears: Liking Elon Musk doesn’t mean y...,liking elon musk doesn t mean you are anythin...,-0.312500,0.687500,positive,0.000,0.342,0.658,0.6369
1,@braveheart034 Hocam Elon Musk twitlerine bin ...,34 hocam elon musk twitlerine bin basar ok g...,0.500000,0.500000,positive,0.000,0.196,0.804,0.2960
2,Elon Musk mission is to leave a dying planet t...,elon musk mission is to leave a dying planet t...,-0.031818,0.450000,negative,0.333,0.000,0.667,-0.6705
3,Volkswagen's boss is adamant that the German g...,volkswagen s boss is adamant that the german g...,0.096429,0.558929,neutral,0.000,0.000,1.000,0.0000
4,RT @TeslaNY: Wedbush: “We view Guillen moving ...,wedbush we view guillen moving to the heavy...,0.058333,0.458333,positive,0.000,0.087,0.913,0.2023
5,.@elonmusk turns down $1 million for the tweet...,turns down 1 million for the tweet he was ...,-0.155556,0.288889,neutral,0.000,0.000,1.000,0.0000
6,RT @lingualandjp: イーロン・マスク氏が、自分の肩書きをCEOからテクノキン...,ceo ...,0.000000,0.000000,neutral,0.000,0.000,1.000,0.0000
7,Liking Elon Musk doesn’t mean you are anything...,liking elon musk doesn t mean you are anything...,-0.312500,0.687500,positive,0.000,0.342,0.658,0.6369
8,RT @aliensosandy: elon musk and grimes baby wh...,elon musk and grimes baby when they ask it ho...,0.250000,0.333333,negative,0.091,0.000,0.909,-0.2500
9,Coinvolgere come Elon Musk https://t.co/pbOaft...,coinvolgere come elon musk,0.000000,0.000000,neutral,0.000,0.000,1.000,0.0000
